In [47]:
import pandas as pd
import numpy as np



import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import datetime
import time
import calendar

In [ ]:
'073p1/073p1_historic.nc'

In [73]:
stn = '073'
startdate = "01/01/2010" # MM/DD/YYYY
enddate = "08/2014"

data_url = 'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/' + stn + 'p1/' + stn + 'p1_historic.nc'
# CDIP Realtime Dataset URL
# data_url = 'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/realtime/' + stn + 'p1_rt.nc'
nc = netCDF4.Dataset(data_url)

OSError: [Errno -90] NetCDF: file not found: b'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/073p1/073p1_historic.nc'

In [72]:
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    naming_authority: edu.ucsd.cdip
    keywords_vocabulary: Global Change Master Directory (GCMD) Earth Science Keywords
    date_created: 2022-01-24T18:21:42Z
    date_issued: 2022-01-24T18:21:42Z
    creator_name: Coastal Data Information Program, SIO/UCSD
    creator_url: http://cdip.ucsd.edu
    creator_email: www@cdip.ucsd.edu
    publisher_name: Coastal Data Information Program, SIO/UCSD
    publisher_url: http://cdip.ucsd.edu
    publisher_email: www@cdip.ucsd.edu
    institution: Scripps Institution of Oceanography, University of California San Diego
    project: Coastal Data Information Program (CDIP)
    processing_level: QA/QC information available at http://cdip.ucsd.edu/documentation
    standard_name_vocabulary: CF Standard Name Table (v29, 08 July 2015)
    Conventions: ACDD-1.3, CF-1.6
    license: These data may be redistributed and used without restriction.
    cdm_data_type

In [30]:
ncTime = nc.variables['sstTime'][:]
timeall = [datetime.datetime.fromtimestamp(t) for t in ncTime] # Convert ncTime variable to datetime stamps
sst = nc.variables['sstSeaSurfaceTemperature'][:] # Make a numpy array of SST variable


# Create variables for Buoy Name and Month Name, to use in plot title
buoyname = nc.variables['metaStationName'][:]
buoytitle = " ".join(buoyname[:-40])

month_name = calendar.month_name[int(startdate[0:2])]
year_num = (startdate[6:10])

TypeError: sequence item 0: expected str instance, numpy.bytes_ found

In [17]:
# Find nearest value in numpy array
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [18]:
def getUnixTimestamp(humanTime,dateFormat):
    unixTimestamp = int(time.mktime(datetime.datetime.strptime(humanTime, dateFormat).timetuple()))
    return unixTimestamp

In [ ]:
unixstart = getUnixTimestamp(startdate,"%m/%d/%Y") 
neareststart = find_nearest(ncTime, unixstart)  # Find the closest unix timestamp
nearIndex = numpy.where(ncTime==neareststart)[0][0]  # Grab the index number of found date

unixend = getUnixTimestamp(enddate,"%m/%d/%Y")
future = find_nearest(ncTime, unixend)  # Find the closest unix timestamp
futureIndex = numpy.where(ncTime==future)[0][0]  # Grab the index number of found date

In [ ]:
# Create figure and specify figure size
fig = plt.figure(figsize=(15,10))

psst = fig.add_subplot(111) 
psst.plot(timeall[nearIndex:futureIndex],sst[nearIndex:futureIndex],'b')

# Set Titles
plt.suptitle(buoytitle, fontsize=30, y=0.99)
title(month_name + " " + year_num, fontsize=20, y=1)

# Set tick parameters
psst.tick_params(axis='y', which='major', labelsize=12, right='off')
psst.tick_params(axis='x', which='major', labelsize=12, top='off')

# Set x-axis tick interval to every 5 days
days = DayLocator(interval=5) 
daysFmt = DateFormatter('%d')
plt.gca().xaxis.set_major_locator(days)
plt.gca().xaxis.set_major_formatter(daysFmt)


# Make a second y-axis, to show values in both Degrees C and Degrees F
psst2 = psst.twinx()

# Set y-axis limits for each plot
psst.set_ylim(5,25)
psst2.set_ylim((5*1.8+32),(25*1.8+32))

# Label each y-axis
psst.set_ylabel('SEA TEMPERATURE (Deg C)', fontsize=18)
psst2.set_ylabel('SEA TEMPERATURE (Deg F)', fontsize=18)

# Label x-axis
psst.set_xlabel('Day', fontsize=18)


# Plot dashed gridlines
psst.grid(b=True, which='major', color='b', linestyle='--')